# [ 5-1. 추정결과 종합 및 보고서 출력 ]

## 1. 기본 모듈 insert

In [12]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

from m00_general_function import df_seperator
from m01_assumption import assumption
from m02_index import index
from m03_funding import funding
from m04_operating_income import operating_income
from m05_operating_cost import operating_cost
from m06_facility_cost import facility_cost

## 2. cashflow.py 파일 생성
* cashflow.py 파일 생성 후 cashflow, balance 생성 코드 붙여넣기

In [1]:
from m07_cashflow import cashflow, balance

In [2]:
cashflow

,date,categoryA,categoryB,categoryC,입금금액,출금금액
0,2023-12-31,자금조달,자기자본,자기자본유입,10000000000,0
1,2023-12-31,자금조달,차입금,차입금유입,10000000000,0
2,2023-12-31,자산매입,자산매입,매입대금지출,0,18000000000
3,2023-12-31,자산매입,매입부수비용,부수비용지출,0,1000000000
4,2023-12-31,객실운영비,수도광열비,Overhead,0,3000000
...,...,...,...,...,...,...
957,2027-01-31,인건비,경영지원팀,정규직,0,10618434
958,2027-01-31,인건비,경영지원팀,임원,0,7865506
959,2027-01-31,인건비,마케팅팀,정규직,0,7078956
960,2027-01-31,인건비,시설관리팀,정규직,0,7078956


In [3]:
balance

,기초현금,입금금액,출금금액,기말현금
2023-12-31,0,20000000000,19094128764,905871236
2024-01-31,905871236,444230000,205812897,1144288339
2024-02-29,1144288339,392080000,191634542,1344733797
2024-03-31,1344733797,305040000,224296397,1425477400
2024-04-30,1425477400,295200000,169210950,1551466450
...,...,...,...,...
2026-09-30,1747490404,301200000,228155785,1820534619
2026-10-31,1820534619,311240000,183068623,1948705996
2026-11-30,1948705996,268800000,171478600,2046027396
2026-12-31,2046027396,451980000,935809678,1562197718


## 3. 피벗테이블 작성
### 3-1. 입금금액과 출금금액 간 차액 계산

In [4]:
cf_table = cashflow.copy()
cf_table

,date,categoryA,categoryB,categoryC,입금금액,출금금액
0,2023-12-31,자금조달,자기자본,자기자본유입,10000000000,0
1,2023-12-31,자금조달,차입금,차입금유입,10000000000,0
2,2023-12-31,자산매입,자산매입,매입대금지출,0,18000000000
3,2023-12-31,자산매입,매입부수비용,부수비용지출,0,1000000000
4,2023-12-31,객실운영비,수도광열비,Overhead,0,3000000
...,...,...,...,...,...,...
957,2027-01-31,인건비,경영지원팀,정규직,0,10618434
958,2027-01-31,인건비,경영지원팀,임원,0,7865506
959,2027-01-31,인건비,마케팅팀,정규직,0,7078956
960,2027-01-31,인건비,시설관리팀,정규직,0,7078956


In [7]:
cf_table['amount'] = cf_table['입금금액'] - cf_table['출금금액']
cf_table = cf_table[['date', 'categoryA', 'categoryB', 'categoryC', 'amount']]

In [8]:
cf_table

,date,categoryA,categoryB,categoryC,amount
0,2023-12-31,자금조달,자기자본,자기자본유입,10000000000
1,2023-12-31,자금조달,차입금,차입금유입,10000000000
2,2023-12-31,자산매입,자산매입,매입대금지출,-18000000000
3,2023-12-31,자산매입,매입부수비용,부수비용지출,-1000000000
4,2023-12-31,객실운영비,수도광열비,Overhead,-3000000
...,...,...,...,...,...
957,2027-01-31,인건비,경영지원팀,정규직,-10618434
958,2027-01-31,인건비,경영지원팀,임원,-7865506
959,2027-01-31,인건비,마케팅팀,정규직,-7078956
960,2027-01-31,인건비,시설관리팀,정규직,-7078956


### 3-2. 피벗테이블 작성

In [9]:
cf_table = cf_table.pivot_table(
    values = 'amount',
    index = 'date',
    columns = ['categoryA', 'categoryB', 'categoryC'],
    aggfunc = 'sum'
).fillna(0).astype(int)

In [10]:
cf_table

categoryA     객실운영비                                                                                         관리운영비            시설관리비                                              운영수입                            인건비                                                                 자금조달               \
categoryB     수도광열비                               예약수수료                       청소세탁비                         관리운영비           대수선공사비                통상수선비                         객실수입                          객실운영팀              경영지원팀               마케팅팀    시설관리팀                  자기자본          차입금   
categoryC  Overhead    TypeA    TypeB    TypeC    TypeA    TypeB    TypeC     TypeA     TypeB     TypeC     광고홍보비     기타운영비  TypeA TypeB TypeC    TypeA    TypeB    TypeC      TypeA      TypeB      TypeC      임시직       정규직       임원       정규직      정규직      임시직      정규직       자기자본유입        차입금유입   
date                                                                                                                                                                                                                                                                                                    
2023-12-31 -3000000        0        0        0        0        0        0         0         0         0 -30000000 -10000000      0     0     0        0        0        0          0          0          0 -5095890 -15287671 -6794520  -9172602 -6115068 -2547945 -6115068  10000000000  10000000000   
2024-01-31 -3000000 -5580000 -6696000 -5270000 -4352400 -5022000 -3952500 -11160000 -13392000 -10540000 -30000000 -10000000      0     0     0 -1116000 -1339200 -1054000  145080000  167400000  131750000 -5081967 -15245901 -6775956  -9147540 -6098360 -2540983 -6098360            0            0   
2024-02-29 -3000000 -4930000 -5916000 -4640000 -3845400 -4437000 -3480000  -9860000 -11832000  -9280000 -30000000 -10000000      0     0     0  -986000 -1183200  -928000  128180000  147900000  116000000 -4754098 -14262295 -6338797  -8557377 -5704918 -2377049 -5704918            0            0   
2024-03-31 -3000000 -4960000 -5952000 -4340000 -2976000 -3571200 -2604000  -9920000 -11904000  -8680000 -10000000 -10000000      0     0     0  -992000 -1190400  -868000   99200000  119040000   86800000 -5081967 -15245901 -6775956  -9147540 -6098360 -2540983 -6098360            0            0   
2024-04-30 -3000000 -4800000 -5760000 -4200000 -2880000 -3456000 -2520000  -9600000 -11520000  -8400000 -10000000 -10000000      0     0     0  -960000 -1152000  -840000   96000000  115200000   84000000 -4918032 -14754098 -6557377  -8852459 -5901639 -2459016 -5901639            0            0   
...             ...      ...      ...      ...      ...      ...      ...       ...       ...       ...       ...       ...    ...   ...   ...      ...      ...      ...        ...        ...        ...      ...       ...      ...       ...      ...      ...      ...          ...          ...   
2026-09-30 -3182700 -5092320 -6110784 -4774050 -2880000 -3456000 -2700000 -10184640 -12221568  -9548100 -10609000 -10609000      0     0     0 -1018464 -1222156  -954810   96000000  115200000   90000000 -5436986 -16310958 -7249315  -9786575 -6524383 -2718493 -6524383            0            0   
2026-10-31 -3182700 -5262064 -6314476 -4933185 -2976000 -3571200 -2790000 -10524128 -12628953  -9866370 -10609000 -10609000      0     0     0 -1052412 -1262895  -986637   99200000  119040000   93000000 -5618219 -16854657 -7490958 -10112794 -6741863 -2809109 -6741863            0            0   
2026-11-30 -3182700 -4455780 -5346936 -4455780 -2520000 -3024000 -2520000  -8911560 -10693872  -8911560 -10609000 -10609000      0     0     0  -891156 -1069387  -891156   84000000  100800000   84000000 -5436986 -16310958 -7249315  -9786575 -6524383 -2718493 -6524383            0            0   
2026-12-31 -3182700 -5919822 -7103786 -5919822 -4352400 -5022000 -4185000 -11839644 -14207572 -11839644 -31827000 -10609000      0     0   

### 3-3. 현금잔액 및 소계 데이터 추가

In [13]:
cf_table[('운영수입', '소계', '')] = operating_income['Total']['객실수입']
cf_table[('객실운영비', '소계', '')] = -operating_cost['객실운영비']['Total']
cf_table[('관리운영비', '소계', '')] = -operating_cost['관리운영비']['Total']
cf_table[('인건비', '소계', '')] = -operating_cost['인건비']['Total']
cf_table[('시설관리비', '소계', '')] = -facility_cost['Total']
cf_table['기초현금'] = balance['기초현금']
cf_table['기말현금'] = balance['기말현금']

In [14]:
cf_table

categoryA     객실운영비                                                                                         관리운영비            시설관리비                                              운영수입                            인건비                                                                 자금조달               \
categoryB     수도광열비                               예약수수료                       청소세탁비                         관리운영비           대수선공사비                통상수선비                         객실수입                          객실운영팀              경영지원팀               마케팅팀    시설관리팀                  자기자본          차입금   
categoryC  Overhead    TypeA    TypeB    TypeC    TypeA    TypeB    TypeC     TypeA     TypeB     TypeC     광고홍보비     기타운영비  TypeA TypeB TypeC    TypeA    TypeB    TypeC      TypeA      TypeB      TypeC      임시직       정규직       임원       정규직      정규직      임시직      정규직       자기자본유입        차입금유입   
date                                                                                                                                                                                                                                                                                                    
2023-12-31 -3000000        0        0        0        0        0        0         0         0         0 -30000000 -10000000      0     0     0        0        0        0          0          0          0 -5095890 -15287671 -6794520  -9172602 -6115068 -2547945 -6115068  10000000000  10000000000   
2024-01-31 -3000000 -5580000 -6696000 -5270000 -4352400 -5022000 -3952500 -11160000 -13392000 -10540000 -30000000 -10000000      0     0     0 -1116000 -1339200 -1054000  145080000  167400000  131750000 -5081967 -15245901 -6775956  -9147540 -6098360 -2540983 -6098360            0            0   
2024-02-29 -3000000 -4930000 -5916000 -4640000 -3845400 -4437000 -3480000  -9860000 -11832000  -9280000 -30000000 -10000000      0     0     0  -986000 -1183200  -928000  128180000  147900000  116000000 -4754098 -14262295 -6338797  -8557377 -5704918 -2377049 -5704918            0            0   
2024-03-31 -3000000 -4960000 -5952000 -4340000 -2976000 -3571200 -2604000  -9920000 -11904000  -8680000 -10000000 -10000000      0     0     0  -992000 -1190400  -868000   99200000  119040000   86800000 -5081967 -15245901 -6775956  -9147540 -6098360 -2540983 -6098360            0            0   
2024-04-30 -3000000 -4800000 -5760000 -4200000 -2880000 -3456000 -2520000  -9600000 -11520000  -8400000 -10000000 -10000000      0     0     0  -960000 -1152000  -840000   96000000  115200000   84000000 -4918032 -14754098 -6557377  -8852459 -5901639 -2459016 -5901639            0            0   
...             ...      ...      ...      ...      ...      ...      ...       ...       ...       ...       ...       ...    ...   ...   ...      ...      ...      ...        ...        ...        ...      ...       ...      ...       ...      ...      ...      ...          ...          ...   
2026-09-30 -3182700 -5092320 -6110784 -4774050 -2880000 -3456000 -2700000 -10184640 -12221568  -9548100 -10609000 -10609000      0     0     0 -1018464 -1222156  -954810   96000000  115200000   90000000 -5436986 -16310958 -7249315  -9786575 -6524383 -2718493 -6524383            0            0   
2026-10-31 -3182700 -5262064 -6314476 -4933185 -2976000 -3571200 -2790000 -10524128 -12628953  -9866370 -10609000 -10609000      0     0     0 -1052412 -1262895  -986637   99200000  119040000   93000000 -5618219 -16854657 -7490958 -10112794 -6741863 -2809109 -6741863            0            0   
2026-11-30 -3182700 -4455780 -5346936 -4455780 -2520000 -3024000 -2520000  -8911560 -10693872  -8911560 -10609000 -10609000      0     0     0  -891156 -1069387  -891156   84000000  100800000   84000000 -5436986 -16310958 -7249315  -9786575 -6524383 -2718493 -6524383            0            0   
2026-12-31 -3182700 -5919822 -7103786 -5919822 -4352400 -5022000 -4185000 -11839644 -14207572 -11839644 -31827000 -10609000      0     0   

### 3-4. 컬럼 순서 정리

In [1]:
cf_table = cf_table[[
    (   '기초현금',       '',         ''),
    (   '자금조달',   '자기자본',   '자기자본유입'),
    (   '자금조달',    '차입금',    '차입금유입'),
    (   '자산매입',   '자산매입',  '매입대금지출'),
    (   '자산매입',   '매입부수비용',  '부수비용지출'),
    (   '운영수입',   '객실수입',     'TypeA'),
    (   '운영수입',   '객실수입',     'TypeB'),
    (   '운영수입',   '객실수입',     'TypeC'),
    (   '운영수입',      '소계',          ''),
    (  '객실운영비',  '예약수수료',    'TypeA'),
    (  '객실운영비',  '예약수수료',    'TypeB'),
    (  '객실운영비',  '예약수수료',    'TypeC'),
    (  '객실운영비',  '청소세탁비',    'TypeA'),
    (  '객실운영비',  '청소세탁비',    'TypeB'),
    (  '객실운영비',  '청소세탁비',    'TypeC'),
    (  '객실운영비',  '수도광열비',    'TypeA'),
    (  '객실운영비',  '수도광열비',    'TypeB'),
    (  '객실운영비',  '수도광열비',    'TypeC'),
    (  '객실운영비',  '수도광열비', 'Overhead'),
    (  '객실운영비',      '소계',         ''),
    (  '관리운영비',  '관리운영비',   '광고홍보비'),
    (  '관리운영비',  '관리운영비',   '기타운영비'),
    (  '관리운영비',      '소계',         ''),
    (    '인건비',  '객실운영팀',      '임시직'),
    (    '인건비',  '객실운영팀',      '정규직'),
    (    '인건비',  '경영지원팀',       '임원'),
    (    '인건비',  '경영지원팀',      '정규직'),
    (    '인건비',   '마케팅팀',      '정규직'),
    (    '인건비',  '시설관리팀',      '임시직'),
    (    '인건비',  '시설관리팀',      '정규직'),
    (    '인건비',      '소계',         ''),
    (  '시설관리비',  '통상수선비',    'TypeA'),
    (  '시설관리비',  '통상수선비',    'TypeB'),
    (  '시설관리비',  '통상수선비',    'TypeC'),
    (  '시설관리비', '대수선공사비',    'TypeA'),
    (  '시설관리비', '대수선공사비',    'TypeB'),
    (  '시설관리비', '대수선공사비',    'TypeC'),
    (  '시설관리비',      '소계',         ''),
    ('차입원리금상환',    '차입금',    '차입금상환'),
    ('차입원리금상환',    '차입금',     '차입이자'),   
    ('자기자본배당', '자기자본배당', '배당금지급'),
    (   '기말현금',       '',         '')
]]

NameError: name 'cf_table' is not defined

In [16]:
cf_table

categoryA         기초현금         자금조달                      자산매입                   운영수입                                     객실운영비                                                                                                   관리운영비                          인건비                                        \
categoryB                      자기자본          차입금         자산매입      매입부수비용       객실수입                               소계    예약수수료                       청소세탁비                        수도광열비                                   소계     관리운영비                  소계    객실운영팀              경영지원팀               마케팅팀   
categoryC                    자기자본유입        차입금유입       매입대금지출      부수비용지출      TypeA      TypeB      TypeC               TypeA    TypeB    TypeC     TypeA     TypeB     TypeC    TypeA    TypeB    TypeC Overhead               광고홍보비     기타운영비                임시직       정규직       임원       정규직      정규직   
date                                                                                                                                                                                                                                                                                                        
2023-12-31           0  10000000000  10000000000 -18000000000 -1000000000          0          0          0          0        0        0        0         0         0         0        0        0        0 -3000000  -3000000 -30000000 -10000000 -40000000 -5095890 -15287671 -6794520  -9172602 -6115068   
2024-01-31   905871236            0            0            0           0  145080000  167400000  131750000  444230000 -4352400 -5022000 -3952500 -11160000 -13392000 -10540000 -5580000 -6696000 -5270000 -3000000 -68964900 -30000000 -10000000 -40000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-02-29  1144288339            0            0            0           0  128180000  147900000  116000000  392080000 -3845400 -4437000 -3480000  -9860000 -11832000  -9280000 -4930000 -5916000 -4640000 -3000000 -61220400 -30000000 -10000000 -40000000 -4754098 -14262295 -6338797  -8557377 -5704918   
2024-03-31  1344733797            0            0            0           0   99200000  119040000   86800000  305040000 -2976000 -3571200 -2604000  -9920000 -11904000  -8680000 -4960000 -5952000 -4340000 -3000000 -57907200 -10000000 -10000000 -20000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-04-30  1425477400            0            0            0           0   96000000  115200000   84000000  295200000 -2880000 -3456000 -2520000  -9600000 -11520000  -8400000 -4800000 -5760000 -4200000 -3000000 -56136000 -10000000 -10000000 -20000000 -4918032 -14754098 -6557377  -8852459 -5901639   
...                ...          ...          ...          ...         ...        ...        ...        ...        ...      ...      ...      ...       ...       ...       ...      ...      ...      ...      ...       ...       ...       ...       ...      ...       ...      ...       ...      ...   
2026-09-30  1747490404            0            0            0           0   96000000  115200000   90000000  301200000 -2880000 -3456000 -2700000 -10184640 -12221568  -9548100 -5092320 -6110784 -4774050 -3182700 -60150162 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-10-31  1820534619            0            0            0           0   99200000  119040000   93000000  311240000 -2976000 -3571200 -2790000 -10524128 -12628953  -9866370 -5262064 -6314476 -4933185 -3182700 -62049076 -10609000 -10609000 -21218000 -5618219 -16854657 -7490958 -10112794 -6741863   
2026-11-30  1948705996            0            0            0           0   84000000  100800000   84000000  268800000 -2520000 -3024000 -2520000  -8911560 -10693872  -8911560 -4455780 -5346936 -4455780 -3182700 -54022188 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-12-31  2046027396            0            0            0           0  145080000  1

## 4. 엑셀파일 출력
### 4-1. 인덱스 표시 수정

In [17]:
cf_table.index[0]

Timestamp('2023-12-31 00:00:00')

In [18]:
cf_table.index = cf_table.index.strftime('%Y-%m-%d')

In [19]:
cf_table.index[0]

'2023-12-31'

### 4-2. 엑셀파일로 출력

In [20]:
cf_table.to_excel("cashflow_table_v1.xlsx")